In [11]:
import openrouteservice as ors


In [12]:
# Example call

loc_1 = (144.9610166, -37.8089811)
loc_2 = (144.9671, -37.8183)
coords = [loc_1, loc_2]

client = ors.Client(base_url="http://localhost:8080/ors")

route = client.directions(coordinates=coords,
                            profile='driving-car',
                            format="geojson")

In [13]:
route

{'type': 'FeatureCollection',
 'metadata': {'attribution': 'openrouteservice.org, OpenStreetMap contributors, tmc - BASt',
  'service': 'routing',
  'timestamp': 1696895723028,
  'query': {'coordinates': [[144.9610166, -37.8089811], [144.9671, -37.8183]],
   'profile': 'driving-car',
   'format': 'geojson'},
  'engine': {'version': '8.0',
   'build_date': '2023-09-20T07:54:24Z',
   'graph_date': '2023-09-20T11:27:39Z'}},
 'bbox': [144.960848, -37.823369, 144.970214, -37.809029],
 'features': [{'bbox': [144.960848, -37.823369, 144.970214, -37.809029],
   'type': 'Feature',
   'properties': {'transfers': 0,
    'fare': 0,
    'segments': [{'distance': 3102.0,
      'duration': 460.2,
      'steps': [{'distance': 375.7,
        'duration': 63.1,
        'type': 11,
        'instruction': 'Head south on Elizabeth Street',
        'name': 'Elizabeth Street',
        'way_points': [0, 10]},
       {'distance': 463.4,
        'duration': 56.5,
        'type': 0,
        'instruction': 'Turn l

In [14]:
# Route length

print(f"Route length is {route['features'][0]['properties']['segments'][0]['distance']} metres")

Route length is 3102.0 metres
